In [2]:
from subsetCounting import SubsetSampling

# Introduction

The original experiment this code was designed for presented a participant with two stimuli to then make a judgement about. There were 34 stimuli divided in two groups.  Using all stimuli and presenting all pairings would require 34x34=1,056 pairwise comparisons (maintaining presentation order).  It was ultimately decided that using only one group of 17 would be sufficient with 17^2=289 pairwise comparisons, and not require an unreasonable number of participants.

It was then decided that a subset of 5 stimuli would make a reasonably sized experiement for a single participant; 5x5=25 pairings with some number of repetitions.  The question then becomes: how many versions of the experiment will be needed to ensure that all 289 comparisons will be used?

The general process that `SubsetSampling` follows is to:
1) create a 'pool' of elements to sample from, 
2) create lists of subsets to serve as mini 'experiments'
3) count all all pairwise samples for each subset

We would ideally like to have equal representation across all pairings.  This might be easy to accomplish for some pool sizes with particular subset sizes, but this is not always guarenteed to be possible (proof needed).  The best we can generally hope for is a fairly even distribution.

# Manual Sampling

You can manually add subsets of your choosing.

In [246]:
p = SubsetSampling(pool_size=5, subset_size=2)
samples = [[1,2],[2,4]]
for s in samples:
    p.updateObservationCounts(s, "Add")
p.info()
p.show()

{
    "poolSize": 5,
    "subsetSize": 2,
    "method": "",
    "params": {},
    "nSubsets": 2,
    "min": 0,
    "max": 1
}
 0, 0, 0, 0, 0
 0, 1, 1, 0, 0
 0, 1, 2, 0, 1
 0, 0, 0, 0, 0
 0, 0, 1, 0, 1


As subsets are added the self pairings along the diagonal will always be higher than pairs of other elements and are not considered for determining the maximum number of observed pairings shown in `info()`.

You can make your own list of subsets to fill the `observationCounts` matrix to your own needs.  Keeping all values (excluding the diagonal) within a difference of 1 or 2 is harder than you may think!

# Random Sampling

Letting chance decide subsets for us can somewhat avoid the issue of over-represnentation.  Each subset can be generated with weights assigned to each index to help guide the pairwise comparisons to a more homogeneous state.

The `buildRandomSubsets` function takes 2 parameters: The size of subset to make and the minumum number required for all pairs of comparisons.

The `trim` function will attempt to remove subsets while still maintaining the minimum pairwise observations provided as a argument.

In [249]:
p = SubsetSampling(10, 5)
p.generateRandomSubsets(1)
p.info()
p.show()

{
    "poolSize": 10,
    "subsetSize": 5,
    "method": "generateRandomSubsets",
    "params": {
        "min_obs": 1
    },
    "nSubsets": 8,
    "min": 1,
    "max": 4
}
 4, 2, 2, 1, 1, 4, 2, 2, 1, 1
 2, 4, 3, 1, 2, 2, 2, 1, 2, 1
 2, 3, 4, 1, 1, 2, 1, 2, 2, 2
 1, 1, 1, 3, 1, 3, 1, 1, 2, 1
 1, 2, 1, 1, 3, 2, 1, 1, 2, 1
 4, 2, 2, 3, 2, 6, 3, 3, 3, 2
 2, 2, 1, 1, 1, 3, 4, 2, 3, 1
 2, 1, 2, 1, 1, 3, 2, 4, 3, 1
 1, 2, 2, 2, 2, 3, 3, 3, 5, 2
 1, 1, 2, 1, 1, 2, 1, 1, 2, 3


Here is the list of subsets that generate the above ObservationMatrix:

In [250]:
p.subsets

[[8, 5, 3, 6, 7],
 [3, 1, 5, 0, 2],
 [8, 6, 2, 9, 1],
 [9, 3, 4, 8, 5],
 [8, 6, 5, 7, 0],
 [2, 7, 1, 8, 4],
 [9, 5, 2, 7, 0],
 [6, 4, 5, 1, 0]]

Using the `n_iters` parameter will find the best list of subsets in that many attemps of random generation.

In [266]:
p = SubsetSampling(10, 5)
p.generateRandomSubsets(1, n_iters=1000)
p.info()
p.show()

{
    "poolSize": 10,
    "subsetSize": 5,
    "method": "generateRandomSubsets",
    "params": {
        "min_obs": 1
    },
    "nSubsets": 6,
    "min": 1,
    "max": 2
}
 3, 1, 1, 1, 2, 2, 1, 1, 2, 1
 1, 3, 1, 2, 1, 1, 2, 1, 2, 1
 1, 1, 3, 2, 1, 2, 1, 1, 1, 2
 1, 2, 2, 3, 2, 1, 1, 1, 1, 1
 2, 1, 1, 2, 3, 1, 1, 2, 1, 1
 2, 1, 2, 1, 1, 3, 2, 1, 1, 1
 1, 2, 1, 1, 1, 2, 3, 2, 1, 1
 1, 1, 1, 1, 2, 1, 2, 3, 1, 2
 2, 2, 1, 1, 1, 1, 1, 1, 3, 2
 1, 1, 2, 1, 1, 1, 1, 2, 2, 3


# Stepped Sampling

Consider all possible ways to sample 5 elements from a set of 17.  This is 17 choose 5, or 6188 and using every single one would evenly saturate the pairwise matrix manytimes over.  It turns out that jumping through this set at even intervals yields good results.

The `generateSubsets` function has two parameters: an initial offset, and the number of jumps to make.  One would need to vary both of these to find an optimal list of subsets.

Using `trim` might be fruitful as well.

In [3]:
p = SubsetSampling(17, 5)
p.generateSteppedSubsets(10, 42)
p.info()
p.show()

{
    "poolSize": 17,
    "subsetSize": 5,
    "method": "generateSteppedSubsets",
    "params": {
        "offset": 10,
        "amount": 42
    },
    "nSubsets": 42,
    "min": 0,
    "max": 8
}
13, 4, 3, 4, 3, 3, 4, 4, 3, 3, 4, 4, 3, 3, 4, 2, 1
 4,13, 3, 4, 5, 2, 4, 3, 1, 3, 5, 3, 4, 2, 4, 2, 3
 3, 3,12, 3, 6, 1, 3, 1, 4, 4, 2, 1, 5, 3, 4, 4, 1
 4, 4, 3,13, 4, 3, 3, 4, 4, 3, 3, 2, 2, 3, 5, 2, 3
 3, 5, 6, 4,14, 2, 5, 1, 2, 3, 4, 3, 6, 0, 8, 2, 2
 3, 2, 1, 3, 2,11, 4, 3, 2, 2, 5, 2, 3, 3, 3, 4, 2
 4, 4, 3, 3, 5, 4,14, 4, 4, 2, 3, 2, 6, 1, 4, 5, 2
 4, 3, 1, 4, 1, 3, 4,11, 4, 2, 4, 0, 2, 3, 3, 2, 4
 3, 1, 4, 4, 2, 2, 4, 4,12, 3, 3, 2, 5, 3, 0, 5, 3
 3, 3, 4, 3, 3, 2, 2, 2, 3,12, 4, 2, 5, 4, 4, 2, 2
 4, 5, 2, 3, 4, 5, 3, 4, 3, 4,14, 2, 5, 2, 2, 6, 2
 4, 3, 1, 2, 3, 2, 2, 0, 2, 2, 2, 8, 1, 3, 4, 0, 1
 3, 4, 5, 2, 6, 3, 6, 2, 5, 5, 5, 1,15, 1, 6, 4, 2
 3, 2, 3, 3, 0, 3, 1, 3, 3, 4, 2, 3, 1,10, 4, 2, 3
 4, 4, 4, 5, 8, 3, 4, 3, 0, 4, 2, 4, 6, 4,16, 3, 6
 2, 2, 4, 2, 2, 4, 5, 2, 5, 2, 6, 0, 

Programmatically trying many values of `offset` and `amount` may find a viable list of subsets depending on ones needs.

Another method one can take is to completely take every possible subset and trim down to the desired minumum value. 

In [4]:
p = SubsetSampling(17, 5)
p.generateSteppedSubsets(10, p.totPoolSubsets)
p.info()
p.show()

{
    "poolSize": 17,
    "subsetSize": 5,
    "method": "generateSteppedSubsets",
    "params": {
        "offset": 10,
        "amount": 6188
    },
    "nSubsets": 6188,
    "min": 455,
    "max": 455
}
1820,455,455,455,455,455,455,455,455,455,455,455,455,455,455,455,455
455,1820,455,455,455,455,455,455,455,455,455,455,455,455,455,455,455
455,455,1820,455,455,455,455,455,455,455,455,455,455,455,455,455,455
455,455,455,1820,455,455,455,455,455,455,455,455,455,455,455,455,455
455,455,455,455,1820,455,455,455,455,455,455,455,455,455,455,455,455
455,455,455,455,455,1820,455,455,455,455,455,455,455,455,455,455,455
455,455,455,455,455,455,1820,455,455,455,455,455,455,455,455,455,455
455,455,455,455,455,455,455,1820,455,455,455,455,455,455,455,455,455
455,455,455,455,455,455,455,455,1820,455,455,455,455,455,455,455,455
455,455,455,455,455,455,455,455,455,1820,455,455,455,455,455,455,455
455,455,455,455,455,455,455,455,455,455,1820,455,455,455,455,455,455
455,455,455,455,455,455,455,455,455

Warning! This can take a minute or two.

In [260]:
p.trim(2)
p.info()
p.show()

{
    "poolSize": 17,
    "subsetSize": 5,
    "method": "generateSteppedSubsets",
    "params": {
        "offset": 10,
        "amount": 6188
    },
    "nSubsets": 48,
    "min": 2,
    "max": 12
}
14, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 3, 6, 9,12
 2,13, 2, 2, 2, 3, 2, 3, 3, 3, 2, 2, 2, 2, 4, 7,11
 2, 2,11, 2, 2, 3, 2, 3, 2, 3, 3, 2, 3, 2, 3, 3, 7
 2, 2, 2,14, 2, 6, 2, 2, 4, 5, 4, 4, 2, 5, 3, 6, 5
 2, 2, 2, 2,10, 2, 3, 2, 2, 4, 2, 2, 2, 3, 3, 3, 4
 2, 3, 3, 6, 2,14, 2, 2, 2, 7, 6, 5, 3, 2, 2, 3, 6
 4, 2, 2, 2, 3, 2,14, 5, 3, 6, 3, 7, 2, 2, 3, 4, 6
 2, 3, 3, 2, 2, 2, 5,12, 3, 2, 3, 4, 4, 2, 2, 3, 6
 2, 3, 2, 4, 2, 2, 3, 3,12, 2, 7, 4, 2, 3, 2, 4, 3
 2, 3, 3, 5, 4, 7, 6, 2, 2,16, 5, 9, 2, 4, 3, 3, 4
 2, 2, 3, 4, 2, 6, 3, 3, 7, 5,15, 5, 4, 5, 3, 2, 4
 2, 2, 2, 4, 2, 5, 7, 4, 4, 9, 5,15, 3, 4, 2, 3, 2
 2, 2, 3, 2, 2, 3, 2, 4, 2, 2, 4, 3,11, 2, 4, 2, 5
 3, 2, 2, 5, 3, 2, 2, 2, 3, 4, 5, 4, 2,12, 2, 3, 4
 6, 4, 3, 3, 3, 2, 3, 2, 2, 3, 3, 2, 4, 2,14, 5, 9
 9, 7, 3, 6, 3, 3, 4, 3, 4, 3, 2, 

As you can see, the `trim` method is not perfect and could use some improvement!  However, trying lots of values for parameters, or multiple random generation attempts can work.

# Store Parameters

Here is one way to generate many subsets to evaluate based on ones needs.

In [263]:
params = {"offset": 2, "amount":4}
p = SubsetSampling(8, 4)
p.generateSteppedSubsets(**params)
p.info()
p.show()

tot 70, step 17.5
[(0, 1, 2, 3), (0, 2, 3, 7), (1, 2, 3, 4), (1, 4, 6, 7)]
{
    "poolSize": 8,
    "subsetSize": 4,
    "method": "generateSteppedSubsets",
    "params": {
        "offset": 2,
        "amount": 4
    },
    "nSubsets": 4,
    "min": 0,
    "max": 3
}
 2, 1, 2, 2, 0, 0, 0, 1
 1, 3, 2, 2, 2, 0, 1, 1
 2, 2, 3, 3, 1, 0, 0, 1
 2, 2, 3, 3, 1, 0, 0, 1
 0, 2, 1, 1, 2, 0, 1, 1
 0, 0, 0, 0, 0, 0, 0, 0
 0, 1, 0, 0, 1, 0, 1, 1
 1, 1, 1, 1, 1, 0, 1, 2


The `SubsetSampling` class will store information that can be used to evaluate how good of a subset list it generated.

In [264]:
p.params

{'poolSize': 8,
 'subsetSize': 4,
 'method': 'generateSteppedSubsets',
 'params': {'offset': 2, 'amount': 4},
 'nSubsets': 4,
 'min': 0,
 'max': 3}